In [1]:
import yt
import astropy.units as u
%matplotlib inline 
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import h5py
from glob import glob

In [2]:
unit_base = {
    "length": (1.0, "pc"),
    "velocity": (100, "cm/s"),
    "mass": (1.0, "Msun"),
}
units_override = dict(length_unit=(1, "pc"), mass_unit=yt.units.mass_sun, velocity_unit = (100, 'cm/s'))
fns = [
    "../data/snapshot_150.hdf5",
    "../data/snapshot_200.hdf5",
    "../data/snapshot_250.hdf5",
    "../data/snapshot_296.hdf5",
]
datadir = '../data/'
paths = glob('../data/' + '/*.hdf5')

In [19]:
profiles = []
labels = []
for i, fn in enumerate(fns):
    ds = yt.load(fn, unit_base = unit_base, unit_system="cgs")  
    ad = ds.all_data()
    MassStar = ad["PartType5", "Masses"].in_units('Msun')
    Coordinates = ad['PartType5', 'Coordinates'].in_units('pc')
    sort = np.argmax(MassStar)
    Center = Coordinates[sort]
    sphere = ds.sphere(Center, (0.05, 'pc'))
    fields = ("PartType0", "radius")
    profiles.append(
    yt.create_profile(
            sphere,('PartType0', 'Temperature'),
            fields=fields,
            weight_field=('gas', 'density'),
            accumulation=True,
        )
    )
p = yt.ProfilePlot.from_profiles(profiles, labels=None)
p
#p.set_unit(("PartType0", "Temperature"), "K")
p.save()

yt : [INFO     ] 2024-04-27 11:55:19,277 ComovingIntegrationOn != 1 or (not found and OmegaLambda is 0.0), so we are turning off Cosmology.
yt : [INFO     ] 2024-04-27 11:55:19,298 Parameters: current_time              = 0.00012483821354781577
yt : [INFO     ] 2024-04-27 11:55:19,298 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-04-27 11:55:19,298 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-04-27 11:55:19,299 Parameters: domain_right_edge         = [8.16 8.16 8.16]
yt : [INFO     ] 2024-04-27 11:55:19,299 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2024-04-27 11:55:19,324 Allocating for 1.214e+07 particles
Loading particle index: 100%|█████████████████| 47/47 [00:00<00:00, 4143.44it/s]
yt : [INFO     ] 2024-04-27 11:55:19,942 ComovingIntegrationOn != 1 or (not found and OmegaLambda is 0.0), so we are turning off Cosmology.
yt : [INFO     ] 2024-04-27 11:55:19,956 Parameters: current_time              = 0.00016645095139

['Multi-data_1d-Profile_Temperature_radius.png']

In [ ]:
fig = plt.figure()
    
grid = AxesGrid(fig,
(0.075, 0.075, 0.85, 0.85),
nrows_ncols=(2, 2),
axes_pad=0.05,
label_mode="L",
share_all=True,
cbar_location="right",
cbar_mode="single",
cbar_size="3%",
cbar_pad="0%",
)

In [ ]:
plot = p.plots[("PartType0", "density")]
plot.figure = fig
plot.axes = grid[i].axes
plot.cax = grid.cbar_axes[i]

# Finally, this actually redraws the plot.
p.render()
p